### Lab 2 - Classification
Matthew Alisangco, Jed Hughes, Chris Dong

## Read data

In [4]:
#  read_table "ISO-8859-1'"
import numpy as np
import pandas as pd

#tweets = pd.read_table('Lab1/clinton_trump_tweets.txt')
#Name, ScreenName, UserID, FollowersCount, FriendsCount, Location, Description, CreatedAt, StatusID, Language, Place, RetweetCount, FavoriteCount, Text

user_cols = ['Name', 'ScreenName', 'UserID', 'FollowersCount', 'FriendsCount', 'Location', 'Description', 'CreatedAt', 'StatusID', 'Language', 'Place', 'RetweetCount', 'FavoriteCount', 'Text']
tweets = pd.read_table('clinton_trump_tweets.txt', encoding ="ISO-8859-1", dtype=str, names= user_cols)

tweets.head(10)

,Name,ScreenName,UserID,FollowersCount,FriendsCount,Location,Description,CreatedAt,StatusID,Language,Place,RetweetCount,FavoriteCount,Text
0,Cebel,Cebel6,1519696717,132,263,"Little Rock, Arkansas",Arkansas Razorback Fan Just trying to be #Unco...,Sat Oct 29 08:10:06 EEST 2016,792232017094119425,en,null,0,1,@NWAJimmy I've read it now though brother. Was...
1,Cookie,Cookiemuffen,109945090,2154,2034,The American South,Got married after college. I don't regret star...,Wed Oct 26 18:44:08 EEST 2016,791304413923213312,en,null,1937,0,RT @wikileaks: New poll puts Pirate Party on c...
2,nolaguy,nolaguy_phd,1450086582,797,1188,NaN,"An LSU Ph.D student living in New Orleans, try...",Sat Oct 29 21:53:29 EEST 2016,792439227090767872,en,null,0,0,@gaystoner821 I think New Orleans spoiled me w...
3,Mark Hager,marksnark,167177185,204,448,Pittsburgh,"Hip, trendy, smart, funny, fit, lobbyist. U? B...",Wed Oct 26 00:33:20 EEST 2016,791029904733331457,en,null,891,0,RT @LOLGOP: ACA needs fixes but know da facts:...
4,Capitalist Creations,aaronjhoddinott,1191022351,775,154,Canada,"Entrepreneur, startup investor, political junk...",Fri Oct 28 05:05:10 EEST 2016,791823089700962304,en,null,7,0,RT @FastCompany: Alphabet shares soar on bette...
5,Josh,BigPapiCarnivor,154393802,646,2985,NaN,"Embrace the feel, induce the euphoria, and jus...",Sat Oct 29 02:34:44 EEST 2016,792147620986839040,en,null,51,0,RT @Pattric: That one follower who tweet you e...
6,David Walling,davidjwalling,106568768,975,2781,"Dallas, TX",Bloodletting secure algorithms close to the bo...,Sat Oct 29 00:16:48 EEST 2016,792112907488079872,en,null,0,0,#infosec #Intel #ACM #IEEE Impacts Haswell mi...
7,robert2266,robert2266,17101060,845,938,The Universe,The Dark Lord,Fri Oct 28 14:41:06 EEST 2016,791968028191711237,en,null,0,0,Hacked e-mails show Clinton campaigns fears ab...
8,Brianna N. Love,Brianna__Loves,17352518,803,1280,#thesea,green blogger|Swells'19 architecture and geosc...,Fri Oct 28 07:32:18 EEST 2016,791860117671124992,en,null,24,0,RT @africaarguments: This year's seen some rem...
9,Polly Ticks,PTicks,2352007196,1422,863,everywhere and nowhere,"a girl interested in the political world, like...",Fri Oct 28 19:01:13 EEST 2016,792033486970454020,en,null,3,0,"RT @HalifaxEditor: Some 40 years later, @HfxRe..."


## Clean data
Remove all Retweets, Remove all users that have Less than 20 tweets

In [7]:
tweetsNoRT = tweets[~tweets.Text.str.startswith('RT')]

tweetsNoRT.head()
#Comments -> {Classifier} -> positive or negative?

tweets.NoRT.GroupBy
        
        


,Name,ScreenName,UserID,FollowersCount,FriendsCount,Location,Description,CreatedAt,StatusID,Language,Place,RetweetCount,FavoriteCount,Text
0,Cebel,Cebel6,1519696717,132,263,"Little Rock, Arkansas",Arkansas Razorback Fan Just trying to be #Unco...,Sat Oct 29 08:10:06 EEST 2016,792232017094119425,en,null,0,1,@NWAJimmy I've read it now though brother. Was...
2,nolaguy,nolaguy_phd,1450086582,797,1188,NaN,"An LSU Ph.D student living in New Orleans, try...",Sat Oct 29 21:53:29 EEST 2016,792439227090767872,en,null,0,0,@gaystoner821 I think New Orleans spoiled me w...
6,David Walling,davidjwalling,106568768,975,2781,"Dallas, TX",Bloodletting secure algorithms close to the bo...,Sat Oct 29 00:16:48 EEST 2016,792112907488079872,en,null,0,0,#infosec #Intel #ACM #IEEE Impacts Haswell mi...
7,robert2266,robert2266,17101060,845,938,The Universe,The Dark Lord,Fri Oct 28 14:41:06 EEST 2016,791968028191711237,en,null,0,0,Hacked e-mails show Clinton campaigns fears ab...
10,neddyo,neddyo,16818809,1400,379,Long Island and beyond...,You should be digging it while it's happening ...,Mon Oct 31 08:06:52 EET 2016,792971077836124160,en,null,0,1,Hulk smash!
